In [1]:
import pandas as pd
data = pd.read_csv(r'/Users/chenyarou/Desktop/IMT 574/IMT574/analysis_scope_final.csv', low_memory=False) 
data.head()

,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,...,EMPLOYER_STATE,EMPLOYER_CITY,NAICS_CODE,LAWFIRM_NAME_BUSINESS_NAME,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PW_WAGE_LEVEL,H_1B_DEPENDENT,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,Certified,2020-02-19,2020-02-26,H-1B,15-1199.00,"Computer Occupations, All Other",Y,1,1,0,...,CA,LA PALMA,541611,LAW OFFICE OF VACHHANI & ASSOCIATES,32.5,Hour,II,N,32.44,Hour
1,Certified,2020-03-12,2020-03-19,H-1B,17-2051.00,Civil Engineers,Y,1,1,0,...,CA,Pasadena,541330,Dornbaum & Peregoy,71500.0,Year,I,N,68162.00,Year
2,Certified,2020-04-19,2020-04-24,H-1B,15-1199.01,Software Quality Assurance Engineers and Testers,Y,1,1,0,...,MA,Southborough,541511,NaN,79934.0,Year,II,Y,79934.00,Year
3,Certified,2020-03-10,2020-03-17,H-1B,13-1111.00,Management Analysts,Y,1,1,0,...,MA,BOSTON,541611,Seyfarth Shaw LLP,165000.0,Year,III,N,112403.00,Year
4,Certified,2020-03-24,2020-03-31,H-1B,15-1131.00,Computer Programmers,Y,1,1,0,...,NC,Durham,54151,"Fakhoury Global Immigration, USA PC",70200.0,Year,II,N,70200.00,Year


### Data processing

1. process null values （公司和律所是否一对一）
2. duplicated rows
3. caculate wage unit from hour to year
4. unconsistent data type

# Data Cleaning

## 1. Drop null values

In [2]:
# drop the rows where the value of 'EMPLOYER_NAME', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY' is null
# since there are not many null values
data = data.dropna(subset=['EMPLOYER_NAME', 'EMPLOYER_STATE', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY'])
# drop the rows where the value of 'H_1B_DEPENDENT' is null
data = data.dropna(subset=['H_1B_DEPENDENT'])
# replace the null value with 'Unknown'
data['LAWFIRM_NAME_BUSINESS_NAME'] = data['LAWFIRM_NAME_BUSINESS_NAME'].fillna('Unknown')

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766079 entries, 0 to 791858
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   CASE_STATUS                 766079 non-null  object 
 1   RECEIVED_DATE               766079 non-null  object 
 2   DECISION_DATE               766079 non-null  object 
 3   VISA_CLASS                  766079 non-null  object 
 4   SOC_CODE                    766079 non-null  object 
 5   SOC_TITLE                   766079 non-null  object 
 6   FULL_TIME_POSITION          766079 non-null  object 
 7   TOTAL_WORKER_POSITIONS      766079 non-null  int64  
 8   NEW_EMPLOYMENT              766079 non-null  int64  
 9   CONTINUED_EMPLOYMENT        766079 non-null  int64  
 10  CHANGE_PREVIOUS_EMPLOYMENT  766079 non-null  int64  
 11  NEW_CONCURRENT_EMPLOYMENT   766079 non-null  int64  
 12  CHANGE_EMPLOYER             766079 non-null  int64  
 13  AMENDED_PETITION   

## 2 Calculate the PW_WAGE_LEVEL based on the United States PW WAGE LEVEL RULES

In [3]:
import numpy as np

# define the Prevailing Wage Level calculation function
def assign_wage_level(row):
    if pd.notnull(row["PW_WAGE_LEVEL"]):  # only process null values
        return row["PW_WAGE_LEVEL"]
    
    prevailing_wage = row["PREVAILING_WAGE"]
    actual_wage = row["WAGE_RATE_OF_PAY_FROM"]
    
    if pd.isnull(prevailing_wage) or pd.isnull(actual_wage):  
        return np.nan

    # Calculate the ratio of salary and prevailing wage
    ratio = actual_wage / prevailing_wage
    
    # Decide the  Prevailing Wage Level based on the OES percentile
    if ratio < 1.17:
        return "I"  # Level 1 (17th percentile)
    elif ratio < 1.34:
        return "II"  # Level 2 (34th percentile)
    elif ratio < 1.5:
        return "III"  # Level 3 (50th percentile)
    else:
        return "IV"  # Level 4 (above 50th percentile)

# fill the null values of PW_WAGE_LEVEL
data["PW_WAGE_LEVEL"] = data.apply(assign_wage_level, axis=1)

# check the result
data["PW_WAGE_LEVEL"].isnull().sum()

np.int64(0)

## 3 Convert the type of RECEIVED_DATE, DECISION_DATE to datetime and calculate the processing days

In [4]:
data["RECEIVED_DATE"] = pd.to_datetime(data["RECEIVED_DATE"], format='%Y-%m-%d', errors='coerce')
data["DECISION_DATE"] = pd.to_datetime(data["DECISION_DATE"], format='%Y-%m-%d', errors='coerce')
data['RECEIVED_YEAR'] = data['RECEIVED_DATE'].dt.year
data['RECEIVED_MONTH'] = data['RECEIVED_DATE'].dt.month
data['DECISION_YEAR'] = data['DECISION_DATE'].dt.year
data['DECISION_MONTH'] = data['DECISION_DATE'].dt.month
data["PROCESSING_DAYS"] = (data["DECISION_DATE"] - data["RECEIVED_DATE"]).dt.days

## 4 Specify and standardize the columns that need to be unified in uppercase

In [5]:
cols_to_upper = ["SOC_TITLE", "EMPLOYER_NAME", "EMPLOYER_CITY", "LAWFIRM_NAME_BUSINESS_NAME"]
data[cols_to_upper] = data[cols_to_upper].apply(lambda x: x.str.upper())

In [6]:
import re

def clean_string(s):
    return re.sub(r'[^\w\s]', '', s)

data['UNIFIED_EMPLOYER_NAME'] = data['EMPLOYER_NAME'].apply(clean_string)
data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'] = data['LAWFIRM_NAME_BUSINESS_NAME'].apply(clean_string)

data['UNIFIED_EMPLOYER_NAME'] = data['UNIFIED_EMPLOYER_NAME'].str.replace(' ', '')
data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'] = data['UNIFIED_LAWFIRM_NAME_BUSINESS_NAME'].str.replace(' ', '')

data.head()

,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,VISA_CLASS,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,...,H_1B_DEPENDENT,PREVAILING_WAGE,PW_UNIT_OF_PAY,RECEIVED_YEAR,RECEIVED_MONTH,DECISION_YEAR,DECISION_MONTH,PROCESSING_DAYS,UNIFIED_EMPLOYER_NAME,UNIFIED_LAWFIRM_NAME_BUSINESS_NAME
0,Certified,2020-02-19,2020-02-26,H-1B,15-1199.00,"COMPUTER OCCUPATIONS, ALL OTHER",Y,1,1,0,...,N,32.44,Hour,2020,2,2020,2,7,FRIENDLYFRANCHISEESCORPORATION,LAWOFFICEOFVACHHANIASSOCIATES
1,Certified,2020-03-12,2020-03-19,H-1B,17-2051.00,CIVIL ENGINEERS,Y,1,1,0,...,N,68162.00,Year,2020,3,2020,3,7,TETRATECHINC,DORNBAUMPEREGOY
2,Certified,2020-04-19,2020-04-24,H-1B,15-1199.01,SOFTWARE QUALITY ASSURANCE ENGINEERS AND TESTERS,Y,1,1,0,...,Y,79934.00,Year,2020,4,2020,4,5,VIRTUSACORPORATION,UNKNOWN
3,Certified,2020-03-10,2020-03-17,H-1B,13-1111.00,MANAGEMENT ANALYSTS,Y,1,1,0,...,N,112403.00,Year,2020,3,2020,3,7,THEBOSTONCONSULTINGGROUPINC,SEYFARTHSHAWLLP
4,Certified,2020-03-24,2020-03-31,H-1B,15-1131.00,COMPUTER PROGRAMMERS,Y,1,1,0,...,N,70200.00,Year,2020,3,2020,3,7,IBMCORPORATION,FAKHOURYGLOBALIMMIGRATIONUSAPC


## 5 Standardize the value of H_1B_DEPENDENT

In [7]:
data['H_1B_DEPENDENT'] = data['H_1B_DEPENDENT'].replace({'Yes': 'Y', 'No': 'N'})

## 6 Standardize the unit of wages

In [8]:
# define the convertion function
def convert_to_yearly_wage(row):
    if row['WAGE_UNIT_OF_PAY'] == 'Hour':
        return row['WAGE_RATE_OF_PAY_FROM'] * 2080  
    elif row['WAGE_UNIT_OF_PAY'] == 'Month':
        return row['WAGE_RATE_OF_PAY_FROM'] * 12  
    elif row['WAGE_UNIT_OF_PAY'] == 'Bi-Weekly':
        return row['WAGE_RATE_OF_PAY_FROM'] * 26  
    elif row['WAGE_UNIT_OF_PAY'] == 'Week':
        return row['WAGE_RATE_OF_PAY_FROM'] * 52  
    else:
        return row['WAGE_RATE_OF_PAY_FROM']  # return directly if it is 'Year' already

def convert_to_yearly_pw(row):
    if row['PW_UNIT_OF_PAY'] == 'Hour':
        return row['PREVAILING_WAGE'] * 2080 
    elif row['PW_UNIT_OF_PAY'] == 'Month':
        return row['PREVAILING_WAGE'] * 12  
    elif row['PW_UNIT_OF_PAY'] == 'Bi-Weekly':
        return row['PREVAILING_WAGE'] * 26  
    elif row['PW_UNIT_OF_PAY'] == 'Week':
        return row['PREVAILING_WAGE'] * 52 
    else:
        return row['PREVAILING_WAGE'] 

# apply the function
data['WAGE_RATE_OF_PAY_FROM'] = data.apply(convert_to_yearly_wage, axis=1)
data['PREVAILING_WAGE'] = data.apply(convert_to_yearly_pw, axis=1)

# update all the unit to 'Year'
data['WAGE_UNIT_OF_PAY'] = 'Year'  
data['PW_UNIT_OF_PAY'] = 'Year'  


data[['WAGE_RATE_OF_PAY_FROM', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY']].head()


,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,67600.0,Year,67475.2,Year
1,71500.0,Year,68162.0,Year
2,79934.0,Year,79934.0,Year
3,165000.0,Year,112403.0,Year
4,70200.0,Year,70200.0,Year


# Train the Baseline Logistic Model

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766079 entries, 0 to 791858
Data columns (total 32 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   CASE_STATUS                         766079 non-null  object        
 1   RECEIVED_DATE                       766079 non-null  datetime64[ns]
 2   DECISION_DATE                       766079 non-null  datetime64[ns]
 3   VISA_CLASS                          766079 non-null  object        
 4   SOC_CODE                            766079 non-null  object        
 5   SOC_TITLE                           766079 non-null  object        
 6   FULL_TIME_POSITION                  766079 non-null  object        
 7   TOTAL_WORKER_POSITIONS              766079 non-null  int64         
 8   NEW_EMPLOYMENT                      766079 non-null  int64         
 9   CONTINUED_EMPLOYMENT                766079 non-null  int64         
 10  CHANGE_PREVIO

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
label_columns = ['VISA_CLASS', 'SOC_CODE', 'SOC_TITLE', 'FULL_TIME_POSITION', 'EMPLOYER_NAME', 'EMPLOYER_STATE', 
                 'EMPLOYER_CITY', 'LAWFIRM_NAME_BUSINESS_NAME', 'WAGE_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'H_1B_DEPENDENT', 
                 'PW_UNIT_OF_PAY', 'UNIFIED_EMPLOYER_NAME', 'UNIFIED_LAWFIRM_NAME_BUSINESS_NAME']

label_encoder = LabelEncoder()

# Conduct Lable Encoding for every categorical feature
for col in label_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))


print(data.dtypes)

CASE_STATUS                                   object
RECEIVED_DATE                         datetime64[ns]
DECISION_DATE                         datetime64[ns]
VISA_CLASS                                     int64
SOC_CODE                                       int64
SOC_TITLE                                      int64
FULL_TIME_POSITION                             int64
TOTAL_WORKER_POSITIONS                         int64
NEW_EMPLOYMENT                                 int64
CONTINUED_EMPLOYMENT                           int64
CHANGE_PREVIOUS_EMPLOYMENT                     int64
NEW_CONCURRENT_EMPLOYMENT                      int64
CHANGE_EMPLOYER                                int64
AMENDED_PETITION                               int64
EMPLOYER_NAME                                  int64
EMPLOYER_STATE                                 int64
EMPLOYER_CITY                                  int64
NAICS_CODE                                     int64
LAWFIRM_NAME_BUSINESS_NAME                    

In [12]:
X = data.drop(columns=['CASE_STATUS', 'RECEIVED_DATE', 'DECISION_DATE'])
y = label_encoder.fit_transform(data['CASE_STATUS'].astype(str))
print(X.dtypes)

VISA_CLASS                              int64
SOC_CODE                                int64
SOC_TITLE                               int64
FULL_TIME_POSITION                      int64
TOTAL_WORKER_POSITIONS                  int64
NEW_EMPLOYMENT                          int64
CONTINUED_EMPLOYMENT                    int64
CHANGE_PREVIOUS_EMPLOYMENT              int64
NEW_CONCURRENT_EMPLOYMENT               int64
CHANGE_EMPLOYER                         int64
AMENDED_PETITION                        int64
EMPLOYER_NAME                           int64
EMPLOYER_STATE                          int64
EMPLOYER_CITY                           int64
NAICS_CODE                              int64
LAWFIRM_NAME_BUSINESS_NAME              int64
WAGE_RATE_OF_PAY_FROM                 float64
WAGE_UNIT_OF_PAY                        int64
PW_WAGE_LEVEL                           int64
H_1B_DEPENDENT                          int64
PREVAILING_WAGE                       float64
PW_UNIT_OF_PAY                    

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

/Users/chenyarou/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline model accuracy: {accuracy:.4f}")


Baseline model accuracy: 0.9906
